In [18]:
import os
import shutil 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import csv
from sklearn.model_selection import train_test_split

In [ ]:
# Task 1 is to make a csv file of imagename, label

In [3]:
# Global Variables
images_folder='Images/'

In [7]:
data=[]
with open('file.csv','w') as file_object:
    csvwriter=csv.writer(file_object)
    csvwriter.writerow(['ImageName','Label'])
    for folder in os.listdir(images_folder):
        for file in os.listdir(os.path.join(images_folder,folder)):
            image_path=os.path.join(images_folder,folder,file)
            data.append([image_path,folder])
    csvwriter.writerows(data)
    

In [37]:
file=pd.read_csv('file.csv')
file['Label']=file['Label'].replace({'Showcase1':0,'Showcase2':1})
X_train,X_test,y_train,y_test=train_test_split(file['ImageName'],file['Label'],random_state=42,test_size=0.2)

train_dataset=tf.data.Dataset.from_tensor_slices((X_train,y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((X_test,y_test))

In [38]:
def read_image(image,label):
  image=tf.io.read_file(image)
  image=tf.image.decode_jpeg(image)
  image=tf.image.resize(image,(224,224))
  label=tf.reshape(label,(-1,1))
  image=tf.cast(image/255.0,dtype=tf.float32)
  target=tf.cast(label,dtype=tf.int64)
  return image,target


train_dataset=train_dataset.map(read_image)
test_dataset=test_dataset.map(read_image)

In [39]:
train_dataset=train_dataset.batch(8)
test_dataset=test_dataset.batch(8)

In [40]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False)
from tensorflow.keras import layers

inputs = layers.Input(shape = (224,224,3), name='inputLayer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name='poolingLayer')(x)
x = layers.Dense(1, name='outputLayer')(x)
outputs = layers.Activation(activation="sigmoid", dtype=tf.float32, name='activationLayer')(x)

model = tf.keras.Model(inputs, outputs, name = "FeatureExtractionModel")

print(model.summary())

Model: "FeatureExtractionModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (InputLayer)     [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 poolingLayer (GlobalAverage  (None, 1280)             0         
 Pooling2D)                                                      
                                                                 
 outputLayer (Dense)         (None, 1)                 1281      
                                                                 
 activationLayer (Activation  (None, 1)                0         
 )                                                               
                                                                 
Total params: 4,050,852
Trainable params: 4,

In [41]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_dataset,epochs=15,validation_data=test_dataset)

Epoch 1/15
13/13 [==============================] - 78s 2s/step - loss: 0.8879 - accuracy: 0.4327 - val_loss: 0.6946 - val_accuracy: 0.4231
Epoch 2/15
13/13 [==============================] - 24s 2s/step - loss: 0.6858 - accuracy: 0.4904 - val_loss: 0.6934 - val_accuracy: 0.4231
Epoch 3/15
13/13 [==============================] - 24s 2s/step - loss: 0.9724 - accuracy: 0.5096 - val_loss: 1.1219 - val_accuracy: 0.4231
Epoch 4/15
13/13 [==============================] - 23s 2s/step - loss: 3.3403 - accuracy: 0.5865 - val_loss: 1.0355 - val_accuracy: 0.4231
Epoch 5/15
13/13 [==============================] - 24s 2s/step - loss: 24.6771 - accuracy: 0.3750 - val_loss: 0.7125 - val_accuracy: 0.4231
Epoch 6/15
13/13 [==============================] - 22s 2s/step - loss: 7.3906 - accuracy: 0.5096 - val_loss: 0.6828 - val_accuracy: 0.5769
Epoch 7/15
13/13 [==============================] - 24s 2s/step - loss: 0.7324 - accuracy: 0.5288 - val_loss: 0.6926 - val_accuracy: 0.5769
Epoch 8/15
13/13 [=